In [1]:
from ucc_ft.checker import ft_check
from ucc_ft.codes import CatStateCode

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


This notebook demonstrates finding a circuit error on a small example. In this case, we consider a cat state preparation circuit. Although not a gadget for an error correcting code, we can still definite its fault-tolerance up to an allowed number of faults in the prepared state. 

In [2]:
num_qubits = 4
max_faults = 2
code = CatStateCode(num_qubits, max_faults)

code.stabilizers()

[stim.PauliString("+ZZ"), stim.PauliString("+_ZZ"), stim.PauliString("+__ZZ")]

In [3]:
circuit = """
OPENQASM 3.0;
include "stdgates.inc";

const uint size = __NUM_QUBITS__;
qubit[size] state;
qubit ancilla;

def cat_prep() {
    bit res = 1;
    while(res != 0) {
        reset state[0];
        res = 0;
        h state[0];

        // QASM loops supported
        for int i in [1:(size-1)] {
            reset state[i];
            cx state[0], state[i];
        }

        // Parity check
        reset ancilla;
        {   // Fails check
            cx state[1], ancilla;
            cx state[2], ancilla;
        }

        /*
        {   //passes check
            cx state[2], ancilla;
            cx state[3], ancilla;
        }*/
        res = measure ancilla;
    }
}
""".replace("__NUM_QUBITS__", str(num_qubits))

res = ft_check(code, circuit, "cat_prep", "prepare", num_ancilla=1)

if not res.is_ft:
    print("Circuit is not fault tolerant.")
    print(f"Error cause: {res.error_cause}")

Precompiling QuantumSE...
   1487.1 ms  ✓ QuantumSE
  1 dependency successfully precompiled in 2 seconds. 65 already precompiled.


shape: (4, 8)
shape: (4, 8)
>>> 0 set_source_line(10)
>>> 0 set_source_line(12)
1
>>> 1 INIT target_qubit=1
>>> 1 set_source_line(14)
2
>>> 3 H target_qubit=1
>>> 3 set_source_line(18)
3
>>> 4 INIT target_qubit=2
>>> 4 set_source_line(19)
4
>>> 8 CNOT target_qubit1=1, target_qubit2=2
>>> 8 set_source_line(18)
5
>>> 9 INIT target_qubit=3
>>> 9 set_source_line(19)
6
>>> 13 CNOT target_qubit1=1, target_qubit2=3
>>> 13 set_source_line(18)
7
>>> 14 INIT target_qubit=4
>>> 14 set_source_line(19)
8
>>> 18 CNOT target_qubit1=1, target_qubit2=4
>>> 18 set_source_line(23)
9
>>> 19 INIT target_qubit=5
>>> 19 set_source_line(25)
10
>>> 23 CNOT target_qubit1=2, target_qubit2=5
>>> 23 set_source_line(26)
11
>>> 27 CNOT target_qubit1=3, target_qubit2=5
>>> 27 set_source_line(34)
12
>>> 28 DestructiveM target_qubit=5, sym_name=DestructiveM_cat_prep_5_1


[ Info: '`bitwuzla -rwl 1`' is used as smt solver for FT_condition case
[ Info: '`bitwuzla -rwl 1`' has solved the problem
[ Info: The assignment that generates the bug has been written to ./_temp_check_FT_condition_.output


>>> Fail!
Circuit is not fault tolerant.
Error cause: None


In [ ]:
from ucc_ft.checker import qasm_to_qprog_source

print(qasm_to_qprog_source(circuit))

# TODO
# R eturn tem_check_FT_condition.output path to python
# Python parses and maps the source lines/error conditions
# Remember a -1 could be initial injected error
# Suppress print output of symbolic execution


__qubit_count = 0
const size = 4;
state = [i + __qubit_count for i in 1:(size)]
__qubit_count += size;
ancilla = __qubit_count + 1 
__qubit_count += 1;
@qprog cat_prep ( ) begin

  set_source_line(10);
  res = bv_val(ctx, 1, 1);
  @repeat begin 
    set_source_line(12);
    INIT(state[( 0 ) + 1 ]);
    res = 0;
    set_source_line(14);
    H(state[( 0 ) + 1 ]);
    for i in (1):((size - 1)) 
      set_source_line(18);
      INIT(state[( i ) + 1 ]);
      set_source_line(19);
      CNOT(state[( 0 ) + 1 ], state[( i ) + 1 ]);
    end
    set_source_line(23);
    INIT(ancilla);
    
      set_source_line(25);
      CNOT(state[( 1 ) + 1 ], ancilla);
      set_source_line(26);
      CNOT(state[( 2 ) + 1 ], ancilla);
    
    set_source_line(34);
    res = DestructiveM(ancilla);
  end :until (res == bv_val(ctx,0,1))
end

